# 循环神经网络

## 1 循环神经网络模块
RNN无法有效应对长时依赖问题，时间跨度较大，所以会丢失较前的信息，LSTM (Long Short Term Memory Networks)和GRU (Gated Recurrent Unit)可以一定程度上解决长时依赖问题，但后来提出的注意力attention机制会更加有效

### 1.1 原始RNN
网络有两个输入：当前时刻$t$的输入$x_t$和$t-1$时刻的隐藏状态$h_{t-1}$
- $x_t$的维度：`(seq, batch, feature)`，分别表示序列长度、批量和输入特征维度
- $h_{t-1}$的维度：`(layers*direction, batch, hidden)`，分别表示层数乘方向(单向为1，双向为2)，批量和输出维度

网络有两个输出：当前时刻$t$的输出$output$和隐藏状态$h_t$
- $output$的维度：`(seq, batch, hidden*direction)`，分别表示序列长度，批量和输出维度乘方向(单向为1，双向为2)
- $h_t$的维度：`(layers*direction, batch, hidden)`，分别表示层数乘方向(单向为1，双向为2)，批量和输出维度

**注意**：$h_{t-1}$和 $h_t$的维度是一致的；在网络初始化时有隐藏状态$h_0$，可以自己指定，如果不指定则默认为0；在单向的RNN中每层只有一个记忆单元，双向的有两个

RNN的内部网络计算公式：
$$h_t=tanh(w_{ih}*x_t+b_{ih}+w_{hh}*h_{t-1}+b_{hh})$$

在PyTorch中使用`nn.RNN(*args, **kargs)`定义，由于参数的形式使用`nn.RNN(input_size=20, hidden_size=50, num_layers=2)`和`nn.RNN(20, 50, 2)`定义的结果一致，参数列表如下：

|参数|功能|
|-|-|
|input_size|输入$x_t$的特征维度|
|hidden_size|输出(隐藏状态)$h_t$的特征维度|
|num_layers|网络层数，默认为**1**|
|nonlinearity|非线性激活函数，默认tanh，可选relu|
|bias|是否使用偏置，默认使用**True**|
|batch_first|决定网络输入维度的顺序，默认顺序为<br>**(seq, batch, feature)**,如果设置为True，<br>顺序变为(batch, seq, feature)|
|dropout|参数接收0~1的数值，在除最后一层外<br>的其他层加dropout层，默认为**0**|
|bidirectional|设置为True表示双向循环神经网络，<br>默认**False**|

### 注意：从该篇起使用0.4版本以来的新规范，Variable和tensor合并后程序有所不同
参考：[Variable和Tensor合并后，PyTorch的代码要怎么改](https://blog.csdn.net/dQCFKyQDXYm3F8rB0/article/details/80105285)
从0.4版本开始程序框架如下：
```python
# torch.device object used throughout this script
device = torch.device("cuda" if use_cuda else "cpu")
model = MyRNN().to(device)
 
# train
total_loss = 0
for input, target in train_loader:
     input, target = input.to(device), target.to(device)
     hidden = input.new_zeros(*h_shape)  # has the same device & dtype as `input`
     ...  # get loss and optimize
     total_loss += loss.item()           # get Python number from 1-element Tensor

# evaluate
with torch.no_grad():                   # operations inside don't track history
     for input, target in test_loader:
```

其他修改部分：
1. 使用`x.detach()`代替Variable中的`x.data`来修改数据
2. 损失函数中使用`loss.item()`代替`loss.data[0]`来获取python数值
3. 使用`tensor.type()`输出tensor的类型
4. `torch.no_grad()`代替`volatile`标志位
5. 可以创建零维向量，如`torch.tensor(2)`，更类似于`numpy.array()`函数
```python
print(torch.tensor(2))  # 只指定数据
print(torch.Tensor(2))  # 既可以指定数据，也可以指定维度来随机数据
# tensor(2)
# tensor(1.00000e-45 *
#      [ 1.4013,  0.0000])
```

In [1]:
import os
import torch
import numpy as np
from torch import nn, optim
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.utils.model_zoo as model_zoo
from torchvision import transforms, datasets, models

In [2]:
basic_rnn = nn.RNN(input_size=20, hidden_size=50, num_layers=2)
print('1.weight_ih的两个权重:', basic_rnn.weight_ih_l0.shape, basic_rnn.weight_ih_l1.shape)
print('  bias_ih的权重:', basic_rnn.bias_ih_l0.shape, basic_rnn.bias_ih_l1.shape)
print('2.weight_hh的两个权重:', basic_rnn.weight_hh_l0.shape, basic_rnn.weight_hh_l1.shape)
print('  bias_hh的权重:', basic_rnn.bias_hh_l0.shape, basic_rnn.bias_hh_l1.shape)

1.weight_ih的两个权重: torch.Size([50, 20]) torch.Size([50, 50])
  bias_ih的权重: torch.Size([50]) torch.Size([50])
2.weight_hh的两个权重: torch.Size([50, 50]) torch.Size([50, 50])
  bias_hh的权重: torch.Size([50]) torch.Size([50])


In [3]:
# 随机化初始输入xt和隐藏状态h0
toy_input = torch.randn([100, 32, 20])            # [seq, batch, feature] 
h_0 = torch.randn([2, 32, 50])                    # [layers*direction, batch, hidden_size]
toy_output, h_n = basic_rnn(toy_input, h_0)
print('1.RNN的输出output:', toy_output.shape)      # [seq, batch, hidden_size*direction] 
print('2.RNN的输出隐层状态:', h_n.shape)           # [layers*direction, batch, hidden_size]

1.RNN的输出output: torch.Size([100, 32, 50])
2.RNN的输出隐层状态: torch.Size([2, 32, 50])


小结：
- 在一层的RNN中，如果是非双向循环的网络，一般只有一个记忆单元，所以隐藏状态的第一维度都是1，双向的网络为2，不同的层有各自的记忆单元，所以隐藏状态$h_t$的第一维度为：**layers*direction**，双向网络direction为2，普通的为1
- 下面程序用于显示进度，以后可能会用，注意print函数中设置`end='\r'`可以覆盖原输出
```python
def show_progress():
    process = '<' + '.'*25 + '>'
    for i in range(1, 26):
        time.sleep(0.5)
        process= process.replace('.', '=', 1)
        print(process, end='\r')
    print('\nfinished!')
# 输出
# <=========================>
# finished!
```

In [48]:

print(torch.Tensor([[1,2],[3,3]]))

tensor([[ 1.,  2.],
        [ 3.,  3.]])
